In [1]:
import pandas as pd
import numpy as np

import os
wd = os.getcwd()
wd

'C:\\Users\\Nils\\Documents\\KIT-TVWL\\Master\\SS 18\\KD Seminar\\Git\\mlp'

In [2]:
df = pd.read_csv("../final_data/complete_data.csv")
df.Timestamp = pd.to_datetime(df.Timestamp)
df = df.drop(["Unnamed: 0"], axis=1)
df.head()

,Timestamp,ID,articleCount,avgSentiment,stdSentiment,25quantileSentiment,50quantileSentiment,75quantileSentiment,maxSentiment,minSentiment,Previous_Day_Return,Next_Day_Return
0,2018-05-02,BASF,156,0.190012,0.236579,0.119930,0.236800,0.323370,0.65250,-0.63234,0.011060,-0.017219
1,2018-05-02,Tesla,510,0.068932,0.173565,-0.070315,0.072627,0.202486,0.64572,-0.50300,0.004093,-0.057051
2,2018-05-02,Google,1901,0.106414,0.192993,-0.005088,0.069690,0.248100,0.98770,-0.55366,0.025714,-0.017464
3,2018-05-02,Allianz,121,-0.052265,0.264638,-0.230230,-0.094850,0.091920,0.58215,-0.66297,0.013549,-0.009181
4,2018-05-02,Airbus,250,0.055327,0.230707,-0.014180,0.030780,0.195380,0.60256,-0.57493,0.007473,0.000612


In [60]:
df["relabeled_returns"] = df["Next_Day_Return"].apply(lambda x: -1 if x < 0 else 1)
df.head()

,Timestamp,ID,articleCount,avgSentiment,stdSentiment,25quantileSentiment,50quantileSentiment,75quantileSentiment,maxSentiment,minSentiment,Previous_Day_Return,Next_Day_Return,relabeled_returns
0,2018-05-02,BASF,156,0.190012,0.236579,0.119930,0.236800,0.323370,0.65250,-0.63234,0.011060,-0.017219,-1
1,2018-05-02,Tesla,510,0.068932,0.173565,-0.070315,0.072627,0.202486,0.64572,-0.50300,0.004093,-0.057051,-1
2,2018-05-02,Google,1901,0.106414,0.192993,-0.005088,0.069690,0.248100,0.98770,-0.55366,0.025714,-0.017464,-1
3,2018-05-02,Allianz,121,-0.052265,0.264638,-0.230230,-0.094850,0.091920,0.58215,-0.66297,0.013549,-0.009181,-1
4,2018-05-02,Airbus,250,0.055327,0.230707,-0.014180,0.030780,0.195380,0.60256,-0.57493,0.007473,0.000612,1


In [61]:
df.shape
nr_days = int(df.shape[0] / 10)
nr_feats = df.shape[1] -4 #(-3 wegen Timestamp, ID, Next_Day_Return, relabeled_returns)
print("Days {}".format(nr_days))
print("Features {}".format(nr_feats))

Days 11
Features 9


In [62]:
df["Timestamp"].value_counts()

2018-05-04    10
2018-05-16    10
2018-05-09    10
2018-05-02    10
2018-05-14    10
2018-05-17    10
2018-05-10    10
2018-05-03    10
2018-05-15    10
2018-05-08    10
2018-05-11    10
Name: Timestamp, dtype: int64

In [79]:
df.tail()

,Timestamp,ID,articleCount,avgSentiment,stdSentiment,25quantileSentiment,50quantileSentiment,75quantileSentiment,maxSentiment,minSentiment,Previous_Day_Return,Next_Day_Return,relabeled_returns
105,2018-05-17,Tesla,297,0.065692,0.231960,-0.087833,0.05566,0.197360,0.747933,-0.64050,-0.006795,-0.027506,-1
106,2018-05-17,Airbus,261,-0.002624,0.262978,-0.179820,0.00000,0.162960,0.605420,-0.54918,0.021861,-0.000714,-1
107,2018-05-17,Allianz,195,-0.048640,0.281950,-0.290885,-0.06882,0.127410,0.565740,-0.58092,0.012064,-0.001565,-1
108,2018-05-17,Telefonica,115,0.088121,0.226507,-0.020130,0.07377,0.235167,0.622090,-0.56565,-0.026461,-0.036134,-1
109,2018-05-17,Samsung,634,0.096998,0.206897,0.000000,0.07874,0.233280,0.636900,-0.89410,-0.009068,0.002022,1


In [65]:
X = []#np.empty((int(nr_days) ,int(nr_feats * 10)))

In [66]:
labels_to_drop = ["Timestamp", "ID", "Next_Day_Return", "relabeled_returns"]
for d in df["Timestamp"].value_counts().index:
    X.append(df[df["Timestamp"] == d].drop(labels_to_drop, axis=1).values.reshape(1,nr_feats*10))
    

In [67]:
X = np.array(X).reshape(int(nr_days), nr_feats*10)
X.shape

(11, 90)

Y for Regression

In [18]:
df_labels = df[["Timestamp","Next_Day_Return"]].copy()
df_labels["Next_Day_Return"].values.reshape(nr_days, 10)

array([[-0.01721889, -0.05705099, -0.01746398, -0.00918132,  0.00061175,
         0.00181067,  0.00350719, -0.0202173 ,  0.        , -0.01004057],
       [ 0.03848335,  0.00773992, -0.0080557 , -0.0065317 ,  0.03332835,
         0.01892551,  0.01262541, -0.02097312,  0.00640083,  0.00847805],
       [ 0.0041417 , -0.01048699, -0.00348867, -0.00353219,  0.00479514,
         0.00792103, -0.00264577,  0.00279818, -0.002593  ,  0.01339733],
       [-0.0328532 , -0.01084348, -0.00882565,  0.02501474,  0.00290647,
         0.00411967,  0.00021751, -0.01747101,  0.00701645,  0.01603135],
       [ 0.01420268,  0.00434028,  0.01365875,  0.00533603, -0.00598168,
        -0.03328691,  0.0099655 ,  0.01445442,  0.01429254,  0.00468751],
       [ 0.00380743, -0.0019508 , -0.005275  , -0.00910477, -0.00260702,
        -0.0013062 , -0.00583092, -0.01306777, -0.00585368, -0.00765923],
       [ 0.00449527,  0.00448108, -0.00126706, -0.00575948, -0.02313133,
        -0.03065852, -0.02483841, -0.00233583

Y for Classification

In [68]:
df_labels = df[["Timestamp","relabeled_returns"]].copy()
df_labels["relabeled_returns"].values.reshape(nr_days, 10)

array([[-1, -1, -1, -1,  1,  1,  1, -1,  1, -1],
       [ 1,  1, -1, -1,  1,  1,  1, -1,  1,  1],
       [ 1, -1, -1, -1,  1,  1, -1,  1, -1,  1],
       [-1, -1, -1,  1,  1,  1,  1, -1,  1,  1],
       [ 1,  1,  1,  1, -1, -1,  1,  1,  1,  1],
       [ 1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [ 1,  1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1,  1, -1, -1,  1,  1,  1],
       [-1,  1,  1,  1, -1,  1,  1, -1, -1,  1],
       [ 1, -1, -1,  1,  1, -1,  1, -1, -1,  1],
       [-1, -1,  1,  1, -1, -1, -1, -1, -1,  1]], dtype=int64)

In [154]:
#df_labels_single = df_labels.groupby("Timestamp").agg("last")

In [155]:
#df_labels_single.head()

,005930.KS,VIV,ALV.DE,TSS,BAYN.DE,TSLA,AIR.DE,AAPL,BAS.DE,BMW.DE
Timestamp,,,,,,,,,,
2018-05-02,0.000000,-0.010665,0.013549,-0.013838,0.003419,0.004093,0.007473,0.043227,0.011060,0.006682
2018-05-03,0.000000,-0.020217,-0.009181,0.008611,0.003507,-0.057051,0.000612,0.001811,-0.017219,-0.010041
2018-05-04,-0.020973,-0.008056,0.008478,0.014659,0.018926,0.033328,0.006401,0.038483,0.012625,-0.006532
2018-05-08,0.013397,-0.010487,-0.003489,0.010593,-0.003532,-0.002646,-0.002593,0.004795,0.002798,0.004142
2018-05-09,-0.032853,-0.017471,-0.008826,0.005824,0.004120,0.016031,-0.010843,0.007016,0.002906,0.000218


In [69]:
#Y = df_labels["Next_Day_Return"].values.reshape(nr_days, 10)#df_labels_single.values
Y = df_labels["relabeled_returns"].values.reshape(nr_days, 10)
Y.shape

(11, 10)

In [104]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LeakyReLU

from keras import backend as K

from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import optimizers
from keras.losses import binary_crossentropy
#from keras.optimizers import SGD

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score

In [71]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [72]:
dims = X.shape[1]
print(dims, 'dims')
print("Building model...")

nb_classes = Y.shape[1]
print(nb_classes, 'classes')

90 dims
Building model...
10 classes


In [73]:
print(X_train.shape)
print(y_train.shape)

(8, 90)
(8, 10)


### Andi

In [76]:
model = Sequential()

# Add layers
model.add(Dense(Y.shape[1], input_shape=(dims,), kernel_initializer='uniform', activation="relu"))
model.add(Dense(Y.shape[1]**2, kernel_initializer='uniform', activation="relu"))
model.add(Dense(Y.shape[1]**2, kernel_initializer='uniform', activation="relu"))
model.add(Dense(Y.shape[1], kernel_initializer='uniform', activation="sigmoid"))

model.compile(loss="binary_crossentropy",
              optimizer="SGD",
              metrics=['accuracy']) # Accuracy performance metric

hist = model.fit(X_train, y_train,
                 epochs=100,
                 batch_size=30,
                 verbose=2,
                 validation_data=(X_val, y_val))

### PREDICTION ###
y_pred = model.predict(X_val)
y_head = np.round(y_pred, 0)

### EVALUATE Classification - MODEL ###

score = model.evaluate(X_val, y_val,verbose=1)


Train on 8 samples, validate on 3 samples
Epoch 1/100
 - 1s - loss: 0.6505 - acc: 0.2500 - val_loss: 0.4616 - val_acc: 0.1333
Epoch 2/100
 - 0s - loss: 0.5118 - acc: 0.2125 - val_loss: 0.2696 - val_acc: 0.1000
Epoch 3/100
 - 0s - loss: 0.3556 - acc: 0.1500 - val_loss: -1.5043e-02 - val_acc: 0.1000
Epoch 4/100
 - 0s - loss: 0.1434 - acc: 0.1250 - val_loss: -2.9672e-01 - val_acc: 0.0333
Epoch 5/100
 - 0s - loss: -7.0619e-02 - acc: 0.1000 - val_loss: -6.3139e-01 - val_acc: 0.0333
Epoch 6/100
 - 0s - loss: -3.0750e-01 - acc: 0.0875 - val_loss: -1.0227e+00 - val_acc: 0.0333
Epoch 7/100
 - 0s - loss: -5.6318e-01 - acc: 0.0875 - val_loss: -1.5514e+00 - val_acc: 0.0333
Epoch 8/100
 - 0s - loss: -8.9050e-01 - acc: 0.0875 - val_loss: -2.4426e+00 - val_acc: 0.0333
Epoch 9/100
 - 0s - loss: -1.4198e+00 - acc: 0.0875 - val_loss: -3.6657e+00 - val_acc: 0.0333
Epoch 10/100
 - 0s - loss: -1.7960e+00 - acc: 0.0875 - val_loss: -2.8827e+00 - val_acc: 0.0333
Epoch 11/100
 - 0s - loss: -2.0651e+00 - acc: 0

Epoch 88/100
 - 0s - loss: -5.0409e+00 - acc: 0.3750 - val_loss: -1.3948e+00 - val_acc: 0.1333
Epoch 89/100
 - 0s - loss: -5.0409e+00 - acc: 0.3750 - val_loss: -1.3915e+00 - val_acc: 0.1333
Epoch 90/100
 - 0s - loss: -5.0409e+00 - acc: 0.3750 - val_loss: -1.3882e+00 - val_acc: 0.1333
Epoch 91/100
 - 0s - loss: -5.0409e+00 - acc: 0.3750 - val_loss: -1.3851e+00 - val_acc: 0.1333
Epoch 92/100
 - 0s - loss: -5.0409e+00 - acc: 0.3750 - val_loss: -1.3820e+00 - val_acc: 0.1333
Epoch 93/100
 - 0s - loss: -5.0409e+00 - acc: 0.3750 - val_loss: -1.3790e+00 - val_acc: 0.1333
Epoch 94/100
 - 0s - loss: -5.0409e+00 - acc: 0.3750 - val_loss: -1.3761e+00 - val_acc: 0.1333
Epoch 95/100
 - 0s - loss: -5.0409e+00 - acc: 0.3750 - val_loss: -1.3732e+00 - val_acc: 0.1333
Epoch 96/100
 - 0s - loss: -5.0409e+00 - acc: 0.3750 - val_loss: -1.3706e+00 - val_acc: 0.1333
Epoch 97/100
 - 0s - loss: -5.0409e+00 - acc: 0.3750 - val_loss: -1.3680e+00 - val_acc: 0.1333
Epoch 98/100
 - 0s - loss: -5.0410e+00 - acc: 0.37

In [77]:
score

[-1.3606109619140625, 0.13333334028720856]

### Nils

In [115]:
model = Sequential()

#Optimizer
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.6, nesterov=True)

#Input Layer
model.add(Dense(nb_classes, input_shape=(dims,), activation="softmax"))#LeakyReLU(alpha=0.3)

#Hidden Layers
model.add(Dense(64, activation=LeakyReLU(alpha=0.3), use_bias=True))
model.add(Dropout(0.4))
model.add(Dense(64, activation="relu", use_bias=True))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu", use_bias=True))
#model.add(Activation('softplus'))

#Output Layer
model.add(Dense(nb_classes, activation="sigmoid"))


model.compile(optimizer="rmsprop", loss='categorical_crossentropy', metrics=["accuracy"])

model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs=10, 
          batch_size=128, verbose=True)

score = model.evaluate(X_val, y_val,verbose=1)

C:\Users\Nils\Anaconda3\lib\site-packages\keras\activations.py:103: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  ).format(identifier=identifier.__class__.__name__))


Train on 8 samples, validate on 3 samples
Epoch 1/10
8/8 [==============================] - 2s 249ms/step - loss: 0.5966 - acc: 0.1250 - val_loss: -7.7008 - val_acc: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 0s 746us/step - loss: 0.4918 - acc: 0.1250 - val_loss: -7.7042 - val_acc: 0.0000e+00
Epoch 3/10
8/8 [==============================] - 0s 750us/step - loss: 0.5463 - acc: 0.1250 - val_loss: -7.7176 - val_acc: 0.3333
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 0.4427 - acc: 0.1250 - val_loss: -7.7354 - val_acc: 0.0000e+00
Epoch 5/10
8/8 [==============================] - 0s 1ms/step - loss: 0.4773 - acc: 0.1250 - val_loss: -7.7428 - val_acc: 0.0000e+00
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 0.4076 - acc: 0.1250 - val_loss: -7.7628 - val_acc: 0.0000e+00
Epoch 7/10
8/8 [==============================] - 0s 999us/step - loss: 0.3442 - acc: 0.0000e+00 - val_loss: -7.7747 - val_acc: 0.0000e+00
Epoch 8/10
8/8 [===

In [95]:
score

[-8.998761177062988, 0.0]

In [135]:
X[0]

array([  5.51000000e+02,   8.21914827e-02,   2.06793140e-01,
        -1.01450000e-02,   4.14600000e-02,   2.13785000e-01,
         7.24430000e-01,  -6.32340000e-01,   0.00000000e+00,
        -2.09731234e-02,  -8.05570083e-03,   8.47804836e-03,
         1.46592056e-02,   1.89255091e-02,   3.33283517e-02,
         6.40083466e-03,   3.84833470e-02,   1.26254066e-02,
        -6.53170186e-03,   3.51000000e+02,  -6.38005924e-05,
         2.54183595e-01,  -1.81920000e-01,  -3.94000000e-02,
         2.19880000e-01,   6.17980000e-01,  -6.99633333e-01,
         0.00000000e+00,  -2.09731234e-02,  -8.05570083e-03,
         8.47804836e-03,   1.46592056e-02,   1.89255091e-02,
         3.33283517e-02,   6.40083466e-03,   3.84833470e-02,
         1.26254066e-02,  -6.53170186e-03,   2.04900000e+03,
         7.03780595e-02,   2.13552659e-01,  -2.62300000e-02,
         3.96700000e-02,   2.10740000e-01,   7.28870000e-01,
        -9.91700000e-01,   0.00000000e+00,  -2.09731234e-02,
        -8.05570083e-03,

In [58]:
E = model.predict(X[0].reshape(1,90))
E

array([[0.0000000e+00, 1.0000000e+00, 1.2261952e-27, 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00, 6.2738419e-01, 1.0000000e+00,
        0.0000000e+00, 1.0000000e+00]], dtype=float32)

In [59]:
Y[0]

array([-0.01721889, -0.05705099, -0.01746398, -0.00918132,  0.00061175,
        0.00181067,  0.00350719, -0.0202173 ,  0.        , -0.01004057])

In [139]:
EY[0]

array([[ 0.04923758,  0.05990248,  0.12029236,  0.14767935,  0.1304228 ,
         0.04514487,  0.12636857,  0.00601043,  0.12278158,  0.1271591 ]])